Student Project:
1. total number of steudents in a file
2. total marks by female/male students
3. total number of students passed/failed. 50+ = pass
4. students enrolled per course
5. total marks achieved per course
6. min/max marks achieved per course
7. avg age of male and female students
8. avg marks students achieved per course


In [52]:
from pyspark import SparkConf, SparkContext

# COMMAND ----------

conf = SparkConf().setAppName("Read File")

sc = SparkContext.getOrCreate(conf=conf)

sd = sc.textFile("StudentData.csv")
header = sd.first()
sdWoHeader = sd.filter(lambda x:x!= header)

totStudents = sdWoHeader.count()
sdWoHeaderSplit = sdWoHeader.map(lambda x:x.split(','))
print("Total students:"+str(totStudents))



Total students:1000


In [53]:
marksByMaleFemale = sdWoHeaderSplit.map(lambda x:(x[1],int(x[5])))
print('Marks by student gender: ')
print( marksByMaleFemale.reduceByKey(lambda x,y:int(x) + int(y)).collect())


Marks by student gender: 
[('Female', 29636), ('Male', 30461)]


Students who passed course => marks greater than 50

In [70]:
marksGT50 = marksByMaleFemale.filter(lambda x: x[1] > 50) 
passCount = marksGT50.count()
failCount = totStudents - passCount
print('Total number of students who passed: ')
print(passCount)
print('Total number of students who failed: ')
print( failCount)


Total number of students who passed: 
630
Total number of students who failed: 
370


Students enrolled per course

In [71]:
studentPerCourse = sdWoHeaderSplit.map(lambda x:(x[3],1))
print('Student enrolled per course: ')
print( studentPerCourse.reduceByKey(lambda x,y:int(x) + int(y)).collect())


Student enrolled per course: 
[('DB', 157), ('Cloud', 192), ('PF', 166), ('MVC', 157), ('OOP', 152), ('DSA', 176)]


Total marks achieved per course

In [72]:
totalMarksPerCourse = sdWoHeaderSplit.map(lambda x:(x[3],x[5]))
print('Total Marks per course : ')
print( totalMarksPerCourse.reduceByKey(lambda x,y:int(x) + int(y)).collect())


Total Marks per course : 
[('DB', 9270), ('Cloud', 11443), ('PF', 9933), ('MVC', 9585), ('OOP', 8916), ('DSA', 10950)]


Min/Max marks achieved per course

In [73]:
minMaxMarks = sdWoHeaderSplit.map(lambda x:(x[3],x[5]))
print('Min Marks per course: ')
print( minMaxMarks.reduceByKey(lambda x,y:(x if x < y else y)).collect())
print('Max Marks per course: ')
print( minMaxMarks.reduceByKey(lambda x,y:(x if x > y else y)).collect())


Min Marks per course: 
[('DB', '20'), ('Cloud', '20'), ('PF', '20'), ('MVC', '22'), ('OOP', '20'), ('DSA', '20')]
Max Marks per course: 
[('DB', '98'), ('Cloud', '99'), ('PF', '99'), ('MVC', '99'), ('OOP', '99'), ('DSA', '99')]


Average marks per course

In [74]:
minMaxMarks = sdWoHeaderSplit.map(lambda x:( x[3], (x[5],1) ) )
averageMarks = minMaxMarks.reduceByKey(lambda x,y: (int(x[0]) + int(y[0]) , int(x[1]) + int(y[1])) )
averageMarks = averageMarks.map(lambda x: (x[0] , (int(x[1][0]) / int(x[1][1]) )))
print(averageMarks.collect())

[('DB', 59.044585987261144), ('Cloud', 59.598958333333336), ('PF', 59.83734939759036), ('MVC', 61.05095541401274), ('OOP', 58.6578947368421), ('DSA', 62.21590909090909)]


Avg age of male and female students

In [79]:
averageAgeByGender = sdWoHeaderSplit.map(lambda x:( x[1], (x[0],1) ) )
averageAge = averageAgeByGender.reduceByKey(lambda x,y: (int(x[0]) + int(y[0]) , int(x[1]) + int(y[1])) )
averageAge = averageAge.map(lambda x: (x[0] , (int(x[1][0]) / int(x[1][1]) )))
print(averageAge.collect())

[('Female', 28.489021956087825), ('Male', 28.52304609218437)]


Avg marks students achieved per course

In [80]:
averageAgeByGender = sdWoHeaderSplit.map(lambda x:( x[3], (x[5],1) ) )
averageAge = averageAgeByGender.reduceByKey(lambda x,y: (int(x[0]) + int(y[0]) , int(x[1]) + int(y[1])) )
averageAge = averageAge.map(lambda x: (x[0] , (int(x[1][0]) / int(x[1][1]) )))
print(averageAge.collect())

[('DB', 59.044585987261144), ('Cloud', 59.598958333333336), ('PF', 59.83734939759036), ('MVC', 61.05095541401274), ('OOP', 58.6578947368421), ('DSA', 62.21590909090909)]
